In [24]:
# Load libraries

import warnings
warnings.filterwarnings('ignore')

#import os

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# for higher resolution
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg','pdf')

# nice format for matplotlib https://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
plt.style.use('bmh')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
#Libraries for Deep Learning Models
from keras.models import Sequential
from keras.layers import Dense
import xgboost as xgb
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
# from keras.wrappers.scikit_learn import KerasClassifier
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD


In [2]:
# load sample dataset (only load the first 300000 rows) (1pts)
data = pd.read_csv('PS2_casestudy_data.csv')
# data = data.drop(data.columns[0], axis=1).head(300000) #try a smaller size first
data = data.drop(data.columns[0], axis=1).head(80000) #try a smaller size first
# data = data.drop(data.columns[0], axis=1).head(50000) #try a smaller size first
data

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,4.518956e+17,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15708,320,50,1722,0,35,-1,79
79996,4.519023e+18,0,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,e2fcccd2,5c5a694b,...,1,0,6616,320,50,576,2,35,-1,32
79997,4.519052e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15703,320,50,1722,0,35,-1,79
79998,4.519317e+18,1,14102100,1002,0,d1a51189,93ab6716,50e219e0,ecad2386,7801e8d9,...,0,0,15547,320,50,1698,3,35,100021,111


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                80000 non-null  float64
 1   click             80000 non-null  int64  
 2   hour              80000 non-null  int64  
 3   C1                80000 non-null  int64  
 4   banner_pos        80000 non-null  int64  
 5   site_id           80000 non-null  object 
 6   site_domain       80000 non-null  object 
 7   site_category     80000 non-null  object 
 8   app_id            80000 non-null  object 
 9   app_domain        80000 non-null  object 
 10  app_category      80000 non-null  object 
 11  device_id         80000 non-null  object 
 12  device_ip         80000 non-null  object 
 13  device_model      80000 non-null  object 
 14  device_type       80000 non-null  int64  
 15  device_conn_type  80000 non-null  int64  
 16  C14               80000 non-null  int64 

In [4]:
#Explore the data (print the first and last 5 rows) (2pts)
print('First five rows:')
print(data.head(5))
print('Last five rows:')
print(data.tail(5))

First five rows:
             id  click      hour    C1  banner_pos   site_id site_domain  \
0  1.000009e+18      0  14102100  1005           0  1fbe01fe    f3845767   
1  1.000017e+19      0  14102100  1005           0  1fbe01fe    f3845767   
2  1.000037e+19      0  14102100  1005           0  1fbe01fe    f3845767   
3  1.000064e+19      0  14102100  1005           0  1fbe01fe    f3845767   
4  1.000068e+19      0  14102100  1005           1  fe8cc448    9166c161   

  site_category    app_id app_domain  ... device_type device_conn_type    C14  \
0      28905ebd  ecad2386   7801e8d9  ...           1                2  15706   
1      28905ebd  ecad2386   7801e8d9  ...           1                0  15704   
2      28905ebd  ecad2386   7801e8d9  ...           1                0  15704   
3      28905ebd  ecad2386   7801e8d9  ...           1                0  15706   
4      0569f928  ecad2386   7801e8d9  ...           1                0  18993   

   C15  C16   C17  C18  C19     C20  C2

In [5]:
#remove nas from dataframe by simply dropping these rows (2pts)
data.dropna()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,4.518956e+17,1,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15708,320,50,1722,0,35,-1,79
79996,4.519023e+18,0,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,e2fcccd2,5c5a694b,...,1,0,6616,320,50,576,2,35,-1,32
79997,4.519052e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15703,320,50,1722,0,35,-1,79
79998,4.519317e+18,1,14102100,1002,0,d1a51189,93ab6716,50e219e0,ecad2386,7801e8d9,...,0,0,15547,320,50,1698,3,35,100021,111


In [6]:
#create Y and X. Y is the "click" column, and X is the other 19 columns (Do not use ['id', 'hour', 'device_id', 'device_ip']) (1pts)
Y = data['click']
X = data.drop(['click', 'id', 'hour', 'device_id', 'device_ip'], axis=1)

In [7]:
#use sklearn one-hot encoder to transform string variables in x to categorical␣columns. make sure to save the data as a sparse matrix. (3pts)

# # Identify the categorical columns
# categorical_cols = ['site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_model']
# 
# # Initialize the OneHotEncoder
# encoder = OneHotEncoder(sparse=True, handle_unknown='ignore')
# 
# # Fit and transform the categorical columns
# X_encoded = encoder.fit_transform(X[categorical_cols])
# 
# # Drop the original categorical columns from X
# X = X.drop(categorical_cols, axis=1)
# 
# # Convert the remaining numerical columns to a sparse matrix
# X_sparse = sparse.csr_matrix(X.values)
# 
# # Combine the numerical and encoded categorical columns
# X_final = sparse.hstack([X_sparse, X_encoded])
# 
# print(f"Shape of final sparse matrix: {X_final.shape}")
# 
# # Check the first few rows to ensure transformation is successful
# print(f"First few rows of the sparse matrix:\n {X_final[:5].todense()}")


string_cols = [i for i in X.columns if X[i].dtype is np.dtype('object')]
print(string_cols)

encoder = OneHotEncoder(sparse=True, handle_unknown = 'ignore')  #sparse=True)#, drop='first')

encoded_x = encoder.fit_transform(X[string_cols])

# encoded_df = pd.DataFrame(encoded_x.toarray(), columns=encoder.get_feature_names_out(string_cols))

unencoded_x = X[[col for col in X.columns if col not in list(string_cols)]].values

X_encoded = np.hstack([unencoded_x, encoded_x.toarray()])



['site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_model']


In [8]:
#do a train-test split. Use the first 90% of the data as training. (1pts)

# Perform train-test split without shuffling to use the first 90% as training data
X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, Y, test_size=0.1, random_state=42, shuffle=False)

print(f"Training set shape: {X_train.shape}, {Y_train.shape}")
print(f"Test set shape: {X_test.shape}, {Y_test.shape}")

Training set shape: (72000, 4635), (72000,)
Test set shape: (8000, 4635), (8000,)



## LASSO and ElasticNet
First use a sklearn LASSO model with alpha=0.005 (regularization penalty) to predict clicking behavior. Report the prediction accuracy. Then use sklearn elastic net with alpha = 0.005 and l1_ratio=0.5 (l1_ratio is a number from 0 to 1 which represents the portion of L1 penalization in the total penalization term)

In [9]:
# initialize and training the model (1pts)
lasso_model = Lasso(alpha=0.005)
lasso_model.fit(X_train, Y_train)

Lasso(alpha=0.005)

In [10]:
# testing the model (2pts)
# Predict on the test set
Y_pred_lasso = lasso_model.predict(X_test)

# Convert predictions to binary outcomes (0 or 1)
Y_pred_lasso_binary = (Y_pred_lasso >= 0.5).astype(int)

# Calculate accuracy
accuracy_lasso = accuracy_score(Y_test, Y_pred_lasso_binary)

# Calculate ROC AUC score
roc_auc_lasso = roc_auc_score(Y_test, Y_pred_lasso)

print(f"LASSO Model Prediction Accuracy: {accuracy_lasso:.4f}")
print(f"LASSO Model ROC AUC Score: {roc_auc_lasso:.4f}")


LASSO Model Prediction Accuracy: 0.8287
LASSO Model ROC AUC Score: 0.6594


In [11]:
# initialize and training the elastic net (2pts)
# Initialize the Elastic Net model with alpha=0.005 and l1_ratio=0.5
elastic_net_model = ElasticNet(alpha=0.005, l1_ratio=0.5)

# Train the Elastic Net model
elastic_net_model.fit(X_train, Y_train)

ElasticNet(alpha=0.005)

In [12]:
# testing the elastic net (2pts)
# Predict on the test set
Y_pred_en = elastic_net_model.predict(X_test)

# Convert predictions to binary outcomes (0 or 1)
Y_pred_en_binary = (Y_pred_en >= 0.5).astype(int)

# Calculate accuracy
accuracy_en = accuracy_score(Y_test, Y_pred_en_binary)

# Calculate ROC AUC score
roc_auc_en = roc_auc_score(Y_test, Y_pred_en)

print(f"Elastic Net Model Prediction Accuracy: {accuracy_en:.4f}")
print(f"Elastic Net Model ROC AUC Score: {roc_auc_en:.4f}")


Elastic Net Model Prediction Accuracy: 0.8294
Elastic Net Model ROC AUC Score: 0.6757


In [13]:
# Compare the two results. What are the differences? What do you think is causing these differences? 2pt
print(f"LASSO Model Prediction Accuracy: {accuracy_lasso:.4f}")
print(f"LASSO Model ROC AUC Score: {roc_auc_lasso:.4f}")
print(f"Elastic Net Model Prediction Accuracy: {accuracy_en:.4f}")
print(f"Elastic Net Model ROC AUC Score: {roc_auc_en:.4f}")

LASSO Model Prediction Accuracy: 0.8287
LASSO Model ROC AUC Score: 0.6594
Elastic Net Model Prediction Accuracy: 0.8294
Elastic Net Model ROC AUC Score: 0.6757


## Decision tree and random forest

In [25]:
# Initialize the Decision Tree model with gini impurity
decision_tree_model = DecisionTreeClassifier(criterion='gini', random_state=42)

# Train the Decision Tree model
decision_tree_model.fit(X_train, Y_train)

# Predict on the test set using predict_proba
Y_pred_dt_proba = decision_tree_model.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, Y_pred_dt_proba)

# Calculate the Youden's J statistic for each threshold
j_scores = tpr - fpr
optimal_idx = np.argmax(j_scores)
optimal_threshold = thresholds[optimal_idx]

# Convert predictions to binary outcomes using the optimal threshold
Y_pred_dt_binary = (Y_pred_dt_proba >= optimal_threshold).astype(int)

# Calculate accuracy
accuracy_dt = accuracy_score(Y_test, Y_pred_dt_binary)
precision_dt = precision_score(Y_test, Y_pred_dt_binary)
recall_dt = recall_score(Y_test, Y_pred_dt_binary)  # Sensitivity
f1_dt = f1_score(Y_test, Y_pred_dt_binary)
conf_matrix_dt = confusion_matrix(Y_test, Y_pred_dt_binary)

# Calculate ROC AUC score
roc_auc_dt = roc_auc_score(Y_test, Y_pred_dt_proba)

# Report number of positive and negative cases in the test set
num_positive_test = (Y_test == 1).sum()
num_negative_test = (Y_test == 0).sum()

print(f"Decision Tree Model Prediction Accuracy: {accuracy_dt:.4f}")
print(f"Decision Tree Model ROC AUC Score: {roc_auc_dt:.4f}")
print(f"Decision Tree Model Precision: {precision_dt:.4f}")
print(f"Decision Tree Model Recall (Sensitivity): {recall_dt:.4f}")
print(f"Decision Tree Model F1-Score: {f1_dt:.4f}")
print(f"Confusion Matrix:\n{conf_matrix_dt}")
print(f"Number of Positive Cases in Test Set: {num_positive_test}")
print(f"Number of Negative Cases in Test Set: {num_negative_test}")
print(f"Optimal Threshold: {optimal_threshold:.4f}")


Decision Tree Model Prediction Accuracy: 0.6624
Decision Tree Model ROC AUC Score: 0.6472
Decision Tree Model Precision: 0.2717
Decision Tree Model Recall (Sensitivity): 0.5797
Decision Tree Model F1-Score: 0.3700
Confusion Matrix:
[[4506 2126]
 [ 575  793]]
Number of Positive Cases in Test Set: 1368
Number of Negative Cases in Test Set: 6632
Optimal Threshold: 0.1515


In [15]:
# do a grid search on the max-depth variable [3,10,None]. (3pts)